<!-- dom:TITLE: Programmation Python  pour les mathématiques -->
# Programmation Python  pour les mathématiques
<!-- dom:AUTHOR: Julien Guillod at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Julien Guillod**, [Sorbonne Université](http://www.sorbonne-universite.fr/),
Licence <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>


L'entier des chapitres est disponible au format
[HTML](https://python.guillod.org//) et [PDF](https://python.guillod.org//python.pdf).
Ce notebook peut également être exécuté sur [mybinder](https://mybinder.org/v2/gh/juguillod/python/master?filepath=chap11.ipynb).






# 11 Cryptographie
<div id="ch:cryptographie"></div>

Depuis le code de César, les méthodes cryptographiques permettant de transmettre des messages secrets ont évoluées en suivant les progrès permettant de les casser. La méthode Vigenère qui est une amélioration du code de César sera étudiée, et nous verrons comment il est possible de casser cette méthode de chiffrement. Ensuite, la méthode de chiffrement RSA qui est une des méthodes de cryptographie asymétrique les plus utilisées actuellement sera introduite.

**Concepts abordés.**

* code Vigenère

* plus grand divisieur commun

* importation de texte

* nombre premier et pseudo-premier

* petit théorème de Fermat

* algorithme d'Eucilde

* algorithme de Rabin-Miller

* optimisation par décorateur

* chiffrement asymétrique RSA





<!-- --- begin exercise --- -->

# Exercice 11.1: Code de Vigenère

Le code de Vigenère consiste à choisir une clef formée par un mot secret (en majuscules) et à le transformer en un vecteur dont les éléments sont les positions de ces lettres dans l'alphabet. Par exemple, "ASECRET" correspond à (0, 18, 4, 2, 17, 4, 19). Pour coder un texte (en majuscules, sans espace ni ponctuation) avec la clef "ASECRET" il suffit de décaler la première lettre par 0, la seconde par 18, la troisième par 4, et ainsi de suite et de recommencer en boucle. Les détails en particulier historiques sont disponibles sur [Wikipédia](https://fr.wikipedia.org/wiki/Chiffre_de_Vigenère).


**a)**
Écrire une fonction `to_int(s)` qui transforme un caractère en sa place dans l'alphabet et écrire la fonction inverse `to_chr(i)`.

<!-- --- begin hint in exercise --- -->

**Indication:**
Voir la documentation des fonctions `ord` et `chr`.

<!-- --- end hint in exercise --- -->



In [1]:
def to_int(s):
    return ord(s)-65

def to_chr(i):
    return chr(i+65)

print(to_int('S'))
print(to_chr(18))

18
S



**b)**
Écrire une fonction `crypt(text, key)` qui chiffre `text` avec le mot secret `key`.



In [2]:
from itertools import cycle

def crypt(text, key):
    code = [str((to_int(char_text) + to_int(char_key))%26) for char_text, char_key in zip(text, cycle(key))]
    return " ".join(code)

text = 'VIVELAFRANCE'
key = 'TRUNG'
crypt(text, key)

'14 25 15 17 17 19 22 11 13 19 21 21'


**c)**
Écrire une fonction permettant de déchiffrer un texte en connaissant la clef.



In [3]:
def decrypt(code, key):
    code_int = list(map(int, code.split(" ")))
    text = [to_chr((num - to_int(char))%26) for num, char in zip(code_int, cycle(key))]
    return "".join(text)

code = '14 25 15 17 17 19 22 11 13 19 21 21'
decrypt(code, key)

'VIVELAFRANCE'



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.2: <span style="color:red">!</span> Casser le code de Vigenère

Charles Babbage a été le premier a casser le code de Vigenère. L'idée est que trois lettres consécutives apparaissant plusieurs fois dans le texte chiffré ont toutes les chances d'être la conséquence du chiffrement des mêmes lettres du message avec les mêmes lettres de la clé. Cela est encore plus probable avec un groupe de quatre lettres. Ainsi l'espacement entre deux mêmes groupes de lettres chiffrées est un multiple de la longueur de la clé. Par exemple, si la répétition d'un groupe est espacée de 28 lettres, puis celle d'un autre de 91, le plus grand diviseur commun de 28 et 91 est 7. Ainsi il est probable que la clé possède 7 lettres. Ensuite connaissant la taille de la clef, il suffit de se baser sur le fait que la lettre "E" est la plus courante en français. Pour que cette stratégie ait une chance de réussir, il faut que la taille du texte soit assez importante.


**a)**
Écrire une fonction permettant de calculer le plus grand diviseur commun (PGDC) entre deux nombre. Écrire une autre fonction permettant de calculer le PGDC entre une liste de nombres.



In [4]:
from functools import reduce

def pgcd(a, b):
    while b != 0:
        r = a % b
        a, b = b, r
    return a

def pgcd_nums(nums):
    return reduce(pgcd, nums)

pgcd_nums([28,91,21,49,77])

7


**b)**
Visiter le site du projet [Gutenberg](http://www.gutenberg.org/browse/languages/fr), choisir votre texte français préféré et télécharger-le au format "Plain Text". Écrire une fonction qui convertisse le texte en majuscules et le débarrasse de toute la ponctuation et autre.

<!-- --- begin hint in exercise --- -->

**Indication:**
Pour convertir un texte en majuscules (en convertissant les accents) et supprimer tous les caractères de ponctuations et autres, il est possible d'utiliser la fonction suivante:

In [5]:
import unicodedata, re

def convert_upper(text):
    # Convert letters to uppercase
    text = text.upper()
    # Un-accent letters
    text = unicodedata.normalize('NFKD', text)
    # Remove special characters
    regex = re.compile('[^a-zA-Z]')
    text = regex.sub('', text)
    return text

original_text = "C'est une matière grave à traiter dans les annales d'un pays comme la France, que l'Histoire des salons de Paris. Depuis une certaine époque, cette histoire se trouve étroitement liée à celle du pays, et surtout aux intrigues toujours attachées aux plans politiques qui si longtemps bouleversèrent le royaume. L'époque de la naissance de la société en France, dans l'acception positive de ce mot, remonte au règne du cardinal de Richelieu. En rappelant la noblesse autour du trône, en lui assignant des fonctions, créant pour elle des charges et des places, dont son orgueil devait jouir, Richelieu donna de la sécurité à la Couronne, sans cesse exposée par les caprices d'un grand seigneur, comme le duc de Bouillon, le duc de Longueville, le duc de Montbazon, et une foule d'autres qui, plus libres dans leurs châteaux, étaient conspirateurs par état et par goût. La réunion de tous ces grands noms autour du trône lui donna plus que de la sécurité, il en doubla la majesté; mais aussi le premier coup fut porté à la noblesse: elle n'eut plus dès-lors de ces grandes entreprises à conduire, qui mettaient en péril à la fois la tête des conspirateurs et le sort de l'État. Richelieu, avec cette justesse de coup d'œil qui lui fit voir le mal sous toutes ses faces, le conjura en appelant la noblesse au Louvre; mais il ne put l'empêcher de conserver ce qui était inhérent à sa nature, toujours portée à l'intrigue et au mouvement. C'est ainsi que, même sous le ministère de Richelieu, on conspirait dans Paris chez les femmes de haute importance, telles que la princesse Palatine, madame de Chevreuse, madame de Longueville, et une foule de femmes toutes-puissantes par leur position dans le monde, leur esprit ou leur beauté... Avides de pouvoir, ces mêmes femmes saisirent, aussitôt qu'elles le comprirent, le moyen que le cardinal lui-même leur avait laissé. Elles régnaient avant dans une ville éloignée, un château-fort habité par des hommes dont le meilleur et le plus agréable n'était souvent qu'un mal-appris; maintenant elles étaient au milieu de Paris, de ce lieu qui, même à cette époque, où il n'était pas embelli par tout le prestige de la Société Parisienne, de cette société qui si longtemps donna partout, en Europe, le modèle du goût et des façons parfaitement nobles et élégantes, formait déjà le parfait gentilhomme. Ce fut alors dans chaque maison particulière qu'il fallut chercher une reine donnant ses lois et dirigeant une opinion. C'est dans les Mémoires du cardinal de Retz, dans ce livre-modèle, qu'on peut reconnaître cette vérité, dans ceux de madame de Motteville. Voyez l'abbé de Gondy lui-même arrivant chez madame de Chevreuse. Suivez-le dans les détours qu'on lui fait parcourir une nuit, pour parvenir jusqu'à la duchesse, lorsqu'il est cependant l'ami de sa fille[1]. Vous le rencontrez ensuite dans les salons à peine organisés, avec M. de Beaufort, M. le duc de Nemours, M. de La Rochefoucauld, et vous êtes admis aux secrets importants de l'époque.... Le salon de madame de Longueville, celui de Mademoiselle, de madame de Lafayette, deviennent comme des clubs à une époque révolutionnaire. Gaston, mannequin de l'abbé de Larivière, dirige tout du Palais-Royal, et la Cour elle-même n'est plus qu'un instrument."

text = convert_upper(original_text)
text

'CESTUNEMATIEREGRAVEATRAITERDANSLESANNALESDUNPAYSCOMMELAFRANCEQUELHISTOIREDESSALONSDEPARISDEPUISUNECERTAINEEPOQUECETTEHISTOIRESETROUVEETROITEMENTLIEEACELLEDUPAYSETSURTOUTAUXINTRIGUESTOUJOURSATTACHEESAUXPLANSPOLITIQUESQUISILONGTEMPSBOULEVERSERENTLEROYAUMELEPOQUEDELANAISSANCEDELASOCIETEENFRANCEDANSLACCEPTIONPOSITIVEDECEMOTREMONTEAUREGNEDUCARDINALDERICHELIEUENRAPPELANTLANOBLESSEAUTOURDUTRONEENLUIASSIGNANTDESFONCTIONSCREANTPOURELLEDESCHARGESETDESPLACESDONTSONORGUEILDEVAITJOUIRRICHELIEUDONNADELASECURITEALACOURONNESANSCESSEEXPOSEEPARLESCAPRICESDUNGRANDSEIGNEURCOMMELEDUCDEBOUILLONLEDUCDELONGUEVILLELEDUCDEMONTBAZONETUNEFOULEDAUTRESQUIPLUSLIBRESDANSLEURSCHATEAUXETAIENTCONSPIRATEURSPARETATETPARGOUTLAREUNIONDETOUSCESGRANDSNOMSAUTOURDUTRONELUIDONNAPLUSQUEDELASECURITEILENDOUBLALAMAJESTEMAISAUSSILEPREMIERCOUPFUTPORTEALANOBLESSEELLENEUTPLUSDESLORSDECESGRANDESENTREPRISESACONDUIREQUIMETTAIENTENPERILALAFOISLATETEDESCONSPIRATEURSETLESORTDELETATRICHELIEUAVECCETTEJUSTESSEDECOUPDILQUILUIFITVOIRLEMALSOUSTOUT

https://www.quora.com/How-can-I-crack-the-Vigenere-cipher-without-knowing-the-key

<!-- --- end hint in exercise --- -->




**c)**
Garder environ 1000 caractères au milieu du texte choisi et chiffrer le avec une clef. Écrire ensuite une fonction permettant de déterminer la longueur de la clef en regardant dans le message chiffré les chaînes identiques de trois caractères ou plus.

<!-- --- begin hint in exercise --- -->

**Indication:**
Former tout d'abord un dictionnaire avec comme clef toutes les occurrences de trois lettres et comme valeur les positions de occurrences. Ensuite déterminer la liste des distances entre les occurrences de trois lettres, puis calculer le PGCD de ces distances. Si ce PGCD est égal à 1 ou est trop petit, alors il réessayer mais avec des mots de longueur plus grande.

<!-- --- end hint in exercise --- -->



In [6]:
def n_letters_distances(crypted_msg, n):
    dic = dict()
    
    for i in range(len(crypted_msg) - n):
        three_letters = crypted_msg[i: i+n]
        if three_letters not in dic:
            dic[three_letters] = []
        dic[three_letters].append(i)
    
    set_distance = set()
    for key in dic.keys():
        if len(dic[key]) == 1:
            continue
        distance_to_1st = dic[key][0]
        for num in dic[key]:
            if num - distance_to_1st != 0:
                set_distance.add(num - distance_to_1st)
    return set_distance

def key_length(code):
    crypted_msg = "".join(map(lambda x: to_chr(int(x)), code.split(" ")))
    n = 3
    while pgcd_nums(n_letters_distances(crypted_msg, n)) <= 3:  # We assume that all keys must be complicated enough to have at least 4 characters
        n += 1
        if n == 10:
            return ("Cannot determine key length")
    return pgcd_nums(n_letters_distances(crypted_msg, n))

keys = ["PARIS", "FRANCE", "ANTIDISESTABLISHMENTARIANISM"]
for key in keys:
    code = crypt(text, key)
    print(f"Key: {key}, real length: {len(key)}, predicted length: {key_length(code)}")

Key: PARIS, real length: 5, predicted length: 5
Key: FRANCE, real length: 6, predicted length: 6
Key: ANTIDISESTABLISHMENTARIANISM, real length: 28, predicted length: 28



**d)**
Écrire une fonction permettant de décrypter un message chiffré en retournant la clef. Essayer de décrypter le texte de votre auteur favori avec cette fonction.

<!-- --- begin hint in exercise --- -->

**Indication:**
Pour trouver la première lettre de la clef, il faut calculer le nombre d'occurrences des 26 lettres de l'alphabet dans le message chiffré qui ont été chiffé avec le premier caractère de la clef. La lettre ayant l'occurrence maximale doit correspondre à la lettre "E". Il suffit ensuite de faire la même chose pour trouver les autres lettres de la clef.

<!-- --- end hint in exercise --- -->



In [7]:
def guess_key(code):
    length = key_length(code)
    letters_list = code.split(" ")
    
    cryptograms = [[]]*length
    for i, letter in enumerate(letters_list):
        if not cryptograms[i % length]:
            cryptograms[i % length] = [0]*26
        cryptograms[i % length][int(letter)] += 1
    key = ""
    for crypto in cryptograms:
        max_where = max(range(len(crypto)), key=lambda i: crypto[i])
        key_char = to_chr((max_where - 4)%26)
        key += key_char
    return key

keys = ["PARIS", "FRANCE", "VIVELAFRANCE", "DAMQUANGTRUNG", "ANTIDISESTABLISHMENTARIANISM"]
for key in keys:
    code = crypt(text, key)
    print(f"Real key: {key}. Guessed key: {guess_key(code)}")

Real key: PARIS. Guessed key: PARIS
Real key: FRANCE. Guessed key: FRANCE
Real key: VIVELAFRANCE. Guessed key: VIVELAFRANCE
Real key: DAMQUANGTRUNG. Guessed key: DAMQUANGTRUNG
Real key: ANTIDISESTABLISHMENTARIANISM. Guessed key: ANTIDISESTABLISHWANTARIAXISM


> ### __The moral of the story is, the shorter and more common the key, the easier it is to crack using basic methods__


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.3: Générer des nombres premiers

La plupart des algorithmes de cryptage actuels sont basés sur l'utilisation de grands nombres premiers. Le but est d'écrire une fonction permettant de générer relativement rapidement des nombres dit pseudo-premiers, c'est-à-dire qui sont premiers avec une probabilité extrêmement grande. La première étape est de générer un grand nombre aléatoire, c'est-à-dire ayant un certain nombre de bits. Ensuite un test de primalité permet de décider si ce nombre est premier ou pas. Si $\pi(n)$ désigne le nombre de nombres premiers $\leq n$ alors, asymptotiquement $\pi(n) \approx \frac{n}{\ln n}$. Ainsi en tirant au hasard un nombre inférieur à $n$ la probabilité qu'il soit premier est d'environ $1/\ln(n)$. Par exemple pour générer un nombre premier de 1024 bits (le minimum garantissant une sécurité raisonnable actuellement), c'est-à-dire de l'ordre de $2^{1024}$, il faut essayer $\ln(2^{1024}) \approx 710$ nombres aléatoires avant d'en trouver un qui soit premier. Vu que tous les nombres pairs sont clairement pas premiers, il suffit d'en tester en moyenne $355$.


**a)**
Écrire un programme permettant de générer un nombre aléatoire impair de $k$ bits, c'est-à-dire compris entre $2^{k-1}$ et $2^{k}$.

<!-- --- begin hint in exercise --- -->

**Indication:**
La façon la plus rapide d'implémenter cela est d'utiliser les opérations sur les bits expliquées [ici](https://docs.python.org/fr/3.6/reference/expressions.html#binary-bitwise-operations).

<!-- --- end hint in exercise --- -->



In [8]:
import random

def random_num(k):
    num = "1"  # Must start with 1 in order to ensure that num is k-bit
    for _ in range(k-2):
        num += random.choice(['0', '1'])
    num += "1"  # Must end with 1 to ensure its oddity
    return int(num, 2)

random_num(100)

930715102681437331946258059675


La façon la plus simple de déterminer si un nombre $n$ est premier est d'essayer de le diviser par tous les nombres entiers $1 < d < n$. Deux remarques permettent de ne pas tester tous les $d$ entre 2 et $n-1$. La première est qu'il est inutile d'essayer les $d$ pairs plus grands que 2. La seconde est qu'il est inutile de tester des nombres plus grands que $\sqrt{n}$.

**b)**
Écrire un algorithme `isprime(n)` permettant de déterminer si un nombre est premier ou pas.



In [39]:
from math import sqrt, ceil

def isprime(n):
    """ Trivial method by trying modulus division with all odd numbers from 3 up to sqrt(n) """
    if n % 2 == 0:
        return False
    elif n == 3:
        return True
    for d in range(3, ceil(sqrt(n))+1 , 2):
        if isprime(d) and n % d == 0:
            return False
    return True

isprime(99971), isprime(121)

(True, False)


**c)**
Écrire une fonction `generate(k,primality)` permettant de générer un nombre premier aléatoire de `k` bits avec le test de primalité `primality`. Tester avec `primality=isprime`. Est-il raisonnable de pouvoir espérer générer un nombre premier de 1024 bit avec cet algorithme ?



In [12]:
import time

def generate(k, primality, N=10000):
    for i in range(N):  # Only try N times, if all fail then return
        num = random_num(k)
        if primality(num):
            return num
    return f"Failed to generate a {k}-bit prime number"

try:
    start = time.time()
    generate(100, isprime)
except KeyboardInterrupt:
    end = time.time()
    print("Interupted kernel after", int(end-start)//60, "minutes because this is taking way to looooong.")

Interupted kernel after 27 minutes because this is taking way to looooong.


> #### There is really no hope for doing this the trivial way with a random odd number as ___small___ as $2^{100}$, let alone $2^{1024}$


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.4: Générer des nombres pseudo-premiers

L'algorithme précédent permettant de générer des nombres premiers étant inutilisable pour générer des grands nombres premiers, une autre approche, probabiliste, est à préconiser. Un test de primalité probabiliste décide qu'un nombre est premier s'il est premier avec une probabilité très grande. Un tel nombre est dit pseudo-premier. Ainsi un test probabiliste peut se tromper et penser qu'un nombre est premier alors qu'en fait il ne l'est pas.

Le test de primalité le plus simple est basé sur le petit théorème de Fermat: si $n$ est premier, alors $a^{n-1}=1 \pmod n$ pour tout $1 \leq a \leq n-1$. Ainsi si on trouve un $a$ tel que $a^{n-1}\neq1 \pmod n$ alors $n$ n'est pas premier. Le test de primalité de Fermat test $N$ valeurs de $a$ choisies aléatoirement et si $a^{n-1}=1 \pmod n$ pour ces $N$ valeurs alors il déclare que $n$ est probablement premier. Les nombres de Carmichael sont pas premiers, mais satisfont $a^{n-1}=1 \pmod n$ pour tout $a$ premier avec $n$. Les premiers nombres de Carmichael sont 561, 1105 et 1729. Si $n$ n'est pas un nombre de Carmichael, alors la probabilité que le test de Fermat se trompe est de $2^{-N}$. En choisissant par exemple $N=128$, on obtient une probabilité de se tromper inférieure à $3\times 10^{-39}$.


**a)**
Écrire une fonction implémentant le test de primalité de Fermat. Utiliser ce test pour générer des nombres premiers aléatoires.

<!-- --- begin hint in exercise --- -->

**Indication:**
Voir la documentation de la fonction `pow` pour une implémentation rapide. Si OpenSSL est installé sur votre ordinateur, il est facile de vérifier si un nombre est premier avec la commande `openssl prime 11` par exemple pour 11.

<!-- --- end hint in exercise --- -->



In [13]:
# The set of Carmichael numbers below is extended to more values.
# Credits: http://oeis.org/wiki/Carmichael_numbers

carmichael = {561, 1105, 1729, 2465, 2821, 6601, 8911, 10585, 15841, 29341, 46657, 52633,
              115921, 162401, 252601, 294409, 314821, 334153, 399001, 410041, 488881, 512461,
              530881, 1024651, 1152271, 1193221, 41041, 62745, 63973, 75361, 101101, 126217,
              172081, 188461, 278545, 340561, 449065, 552721, 656601, 658801, 670033, 748657,
              825265, 838201, 852841, 997633, 1033669, 1050985, 1082809, 1569457}

def isprime_fermat(n, N=128):
    for _ in range(N):
        a = random.randint(2, n-1)
        while a in carmichael or a%2 == 0:
            a = random.randint(2, n-1)
        if pow(a, n-1, n) != 1:
            return False
    return True

In [14]:
num = generate(1024, isprime_fermat)
num

173659750193959872944071826101779364909174669679191868776315564294108816866361963204711337252826487489495342997847284157024656725340703706336389819064160279346213726708227768268437461074119179619038709760174758057169962967610102070727219007145111148917870531951932891718156400716104444768754413956376907876963

In [15]:
!openssl prime {num}

F74CB8AF78DD7A2B7E49D22F1D4641C5C6968F283772FAC190C52725DC08734A15DC1932C7AE02EEE775E8620988F529EDC0F8E6EF39A2C05423AFDDD34F474AAEA38C4E232718600B8AC9D5ACD0E38DD43A74945DBADD1CD3DE4305DBF0E9A7782DA1365BFAA7E2B24048A5132BC30E4F7516E1C4407EA4CB10DE4D0EA20A63 (173659750193959872944071826101779364909174669679191868776315564294108816866361963204711337252826487489495342997847284157024656725340703706336389819064160279346213726708227768268437461074119179619038709760174758057169962967610102070727219007145111148917870531951932891718156400716104444768754413956376907876963) is prime


In [16]:
%%timeit
generate(1024, isprime_fermat)

The slowest run took 8.53 times longer than the fastest. This could mean that an intermediate result is being cached.
2.88 s ± 2.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)



**b)**
<span style="color:red">!</span> Améliorer la rapidité du l'algorithme précédent en testant d'abord si $n$ est divisible par les nombres premiers inférieurs à 1000 avant d'appliquer le test de Fermat.



In [19]:
def primes_lte(n):
    primes = {x for x in range(3, n+1, 2) if isprime(x)}
    primes.add(2)
    return primes

def isprime_fermat_faster(n, N=128):
    for prime in primes_lt_1000:
        if n % prime == 0:
            return False
        
    maxi = max(primes_lt_1000)
    for _ in range(N):
        a = random.randint(maxi, n-1)
        while a in carmichael or a%2 == 0:
            a = random.randint(2, n-1)
        if pow(a, n-1, n) != 1:
            return False
    return True

In [18]:
primes_lt_1000 = primes_lte(1000)

In [19]:
%%timeit
generate(1024, isprime_fermat_faster)

The slowest run took 4.95 times longer than the fastest. This could mean that an intermediate result is being cached.
1.21 s ± 731 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



Le test de primalité de Fermat permet de générer de grands nombres pseudo-premiers avec une bonne probabilité d'être effectivement premier. Le problème principal vient du fait de l'existence des nombres de Carmichael qui sont exclus de cette probabilité. Le test de primalité de Miller-Rabin permet d'éviter se problème.

**c)**
<span style="color:red">!!</span> Comprendre et implémenter la méthode Rabin-Miller expliquée en détails sur [Wikipédia](https://fr.wikipedia.org/wiki/Test_de_primalité_de_Miller-Rabin).



In [20]:
def isprime_miller_rabin(n, k=10):
    s, d = 0, n-1
    while d%2 == 0:
        s += 1
        d >>= 1  # bitwise equivalent of dividing by 2
    
    tested_bases = set()
    for _ in range(k):  # k times executing the composite test for n to be very certain of the primality of n
        a = random.randint(2, n-1)
        while a in tested_bases:
            a = random.randint(2, n-1)
        tested_bases.add(a)
        if pow(a, d, n) in {1, n-1}:
            continue
        continue_loop = False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                continue_loop = True
                break
        if continue_loop:
            continue
        return False
    return True

In [21]:
num = generate(1024, isprime_miller_rabin)
num

161512810805158221529283332276441303215414939853876178505174005015263749359399007192085625874692507786417186371245065570043791572806043734068449916073831559238644939378033171112616202784041212818072910183771048746084187915884414607633760284744078520343633251705610722661333852051271895469569132035288830165297

In [22]:
!openssl prime {num}

E6007B06506E42320F779B0A6EECC16B34D2CD4AD6CDF5C907E21889452AE3A0B570AC628CB610DAB7A9FF1997D00497C5401DD1BDE13B349D6CD860BBED2A6B4B234AA7382F640B7049293BFECEF609A62AF6B53DD78B99CF7F7A94DDB89D55930F054B4D99F393CF7F9AA80D94381C34DDEC5B77EEE0841910A4B767EDC131 (161512810805158221529283332276441303215414939853876178505174005015263749359399007192085625874692507786417186371245065570043791572806043734068449916073831559238644939378033171112616202784041212818072910183771048746084187915884414607633760284744078520343633251705610722661333852051271895469569132035288830165297) is prime


In [23]:
%%timeit
generate(1024, isprime_miller_rabin)

The slowest run took 10.02 times longer than the fastest. This could mean that an intermediate result is being cached.
2.96 s ± 2.26 s per loop (mean ± std. dev. of 7 runs, 1 loop each)



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.5: Chiffrement RSA

L'algorithme RSA des noms de Ronald Rivest, Adi Shamir et Leonard Adleman qui l'ont inventé en 1983 est un des algorithmes de cryptographie asymétrique les plus utilisé encore actuellement. Un chiffrement asymétrique permet de transmettre un message crypté à une personne A sans avoir eu auparavant besoin de transmettre une clef secrète à la personne B qui chiffre le message. La création par A d'une clef publique suffit à B pour chiffrer le message et pour que A puisse le déchiffrer avec sa clef privé. Il y a trois grandes étapes dans l'algorithme RSA:

**Création des clefs.**
La personne A voulant recevoir un message secret, choisit deux très grands nombres premiers $p$ et $q$ qu'il garde secret. Ensuite elle calcule $n=pq$ et l'indicatrice d'Euler $\varphi(n)=(p-1)(q-1)$ qui compte le nombre d'entiers compris entre 1 et $n$ qui sont premiers avec $n$. Puis elle choisit un exposant de chiffrement $e$ qui est premier avec $\varphi(n)$. La clef publique de la personne A est donné par le couple $(n,e)$. Finalement, la personne A calcul l'exposant de déchiffrement $d$ qui est l'inverse de $e$ modulo $\varphi(n)$, *i.e.* tel que $ed = 1 \pmod{\varphi(n)}$. La clef privée de A est $(p,q,d)$.

**Chiffrement du message.**
Pour chiffrer son message, la personne B le transforme tout d'abord en un nombre entier $M < n$. Le message chiffré est alors donné par

$$
C = M^e \pmod n \,.
$$

**Déchiffrement du message.**
Le message chiffré $C$ est alors transmis à A. Pour le déchiffrer, A calcule

$$
M = C^d \pmod n \,
$$

qui est à nouveau le message original.

**Notice.**

A noter que les nombres premiers $p$ et $q$ doivent être vraiment aléatoires, sinon il est "facile" de deviner leurs valeurs. Les nombres aléatoires générés par le module `random` le sont avec l'algorithme de Mersenne Twister. Cet algorithme n'est pas considéré comme cryptographiquement sur dans le sens où une observation d'environs un millier de nombres générés aléatoirement par cet algorithme suffit à prédire toutes les itérations futures. Pour générer des nombres aléatoires cryptographiquement sûrs il faudrait utiliser le module [`secrets`](https://docs.python.org/3.6/library/secrets.html).




**a)**
Montrer mathématiquement que le message déchiffré correspond bien au message original.

<!-- --- begin hint in exercise --- -->

**Indication:**
Si $a = b \pmod{\varphi(n)}$ et $M$ est premier avec $n$, alors $M^a = M^b \pmod n$.

<!-- --- end hint in exercise --- -->



> In fact, based on Euler's theorem, $M$ must also be co-prime with $n$. Indeed, if $C = M^e \pmod{n}$ then $C^d = (M^e)^d = M^{ed} \pmod{n}$. Since $ed = 1 \pmod{\varphi (n)}$, $M$ co-prime with $n$ we have finally $M^{ed}=M \pmod {n}$.


**b)**
Donné $e$ et $\varphi(n)$ écrire une fonction `bezout(e, phi)` permettant de déterminer $d$ tel que $ed = 1 \pmod{\varphi(n)}$.

<!-- --- begin hint in exercise --- -->

**Indication:**
Utiliser l'algorithme d'Euclide généralisé qui permet de déterminer le PGCD $g$ entre deux nombre $a$ et $b$ ainsi que $x$ et $y$ satisfaisant $ax+by=g$.

<!-- --- end hint in exercise --- -->



In [24]:
def bezout(e, phi):
    s, s_ = 0, 1
    t, t_ = 1, 0
    r, r_ = phi, e
    
    while r != 0:
        q = r_ // r
        r_, r = r, r_ - q*r
        s_, s = s, s_ - q*s
        t_, t = t, t_ - q*t
    return s_

In [25]:
e, phi = 1351, 643
(bezout(e, phi) * e) % phi

1


**c)**
Écrire un algorithme `generate_keys(length)` qui génère des nombres premiers $p$ et $q$ tels que $n$ ait `length` bits, puis détermine $\varphi(n)$, $e$ et $d$ et enfin retourne la clef publique $(n,e)$ et la clef privée $(p,q,d)$.



In [26]:
import secrets

def generate_keys(length):
    def random_prime(bits):
        min_value = ceil(2**(bits-0.5))
        prime = secrets.randbits(bits)
        while prime < min_value or not isprime_miller_rabin(prime):
            prime = secrets.randbits(bits)
        return prime
    
    bits_p = secrets.choice(range(length//2 - 10, length//2 + 10))  # randomly choose a range for p's bits
    bits_q = length - bits_p
    p = random_prime(bits_p)
    q = random_prime(bits_q)
    n = p*q
    e = 65537
    phi = (p-1)*(q-1)
    d = bezout(e, phi) if bezout(e, phi) > 0 else phi + bezout(e, phi)  # Should be a positive integer
    
    return (n, e), (p, q, d)

In [27]:
length = 1024
public, private = generate_keys(length)
print("Public key:", public)
print("Private key:", private)

Public key: (102879710770752511444645833011165696651619809409491556295029570602622028588515675930724362634624896351441630621506341531211272436681854793154606165896207790041954416913355391419643914918797618398109799014010931855695176217558393594050253975370232155523911584660100880097160695246305759491396478855962313664689, 65537)
Private key: (79563557840776827903565737400968911552647625307192266548999184353943670464685632026945084663720144251159361004827848299962554503752610900101658946145770817, 1293050657395640084328044917968166196339015056822260360955483553779516366396198317701676056519022318239979406312719222733835422685370445179884973796802417, 44178764670663715604722029974415005277423994021870407080138199878993419756268926983511233923222736134071163016022292288522191131506438186730397810806368517930378109939801855401198146020012241708454526590940672114387451626365242287279016559120153407419391811060195436769803350806644845587338005187189045113857)


In [28]:
n, e = public
p, q, d = private

In [29]:
p*q == n

True

In [30]:
(e*d) % ((p-1)*(q-1))

1

In [34]:
len(bin(n)) - 2  # bin(n) returns a str typed binary value of n which includes '0b' at the beginning, therefore must be excluded to know the exact number of bits of n

1024


**d)**
En choisissant d'encoder chaque caractère sur 8 bits, une chaine de caractère de longueur $\ell$ s'écrit comme une liste $(a_0,a_1,\dots a_\ell)$ avec chaque $0 \leq a_i \leq 255$. Cette liste peut être convertie en un entier $k$ de la façon suivante:

$$
k = \sum_{i=0}^\ell a_i 256^i
$$

Écrire une fonction `toint` et une fonction `tostr` permettant respectivement de convertir une chaîne de caractères en cet entier et inversement.



In [1]:
def toint(string):
    crypted_msg = 0
    for i, char in enumerate(string):
        crypted_msg += ord(char)*256**i
    return crypted_msg

code = toint("I am legend is such a great movie starring Will Smith. I love watching him.")
code

752543656598430791546212673016081644515029171583346043640677252474922246724244942692995773015219717088247296046260360694581531473085574139295866474835470801196647884988166726295625

In [2]:
def tostr(num):
    msg = ""
    while num // 256 >= 0:
        a = 0
        while (num - a) % 256 != 0:
            a += 1
        msg += chr(a)
        num //= 256
        if num == 0:
            break
    return msg

tostr(code)

'I am legend is such a great movie starring Will Smith. I love watching him.'


**e)**
Écrire une fonction pour chiffrer un texte avec une clef publique et une autre permettant de le déchiffrer avec la clef privée. Pour cela il faut s'assurer que le texte soit convertible en un entier inférieur à $n$, sinon il faut le découper en blocs et les chiffrer séparément.



In [3]:
def break_msg(msg, k):
    """ Break the string into k even parts (in terms of number of characters) """
    q = len(msg) // k
    r = len(msg) % k
    return [msg[i*q : (i+1)*q + 1] if i<r else msg[i*q : (i+1)*q] for i in range(k)]

msg = 'I am legend is such a great movie starring Will Smith. I love watching him.'
break_msg(msg, 5)

['I am legend is ',
 'such a great mo',
 'vie starring Wi',
 'll Smith. I lov',
 'e watching him.']

In [5]:
public = (102879710770752511444645833011165696651619809409491556295029570602622028588515675930724362634624896351441630621506341531211272436681854793154606165896207790041954416913355391419643914918797618398109799014010931855695176217558393594050253975370232155523911584660100880097160695246305759491396478855962313664689,
          65537)
private = (79563557840776827903565737400968911552647625307192266548999184353943670464685632026945084663720144251159361004827848299962554503752610900101658946145770817,
           1293050657395640084328044917968166196339015056822260360955483553779516366396198317701676056519022318239979406312719222733835422685370445179884973796802417, 44178764670663715604722029974415005277423994021870407080138199878993419756268926983511233923222736134071163016022292288522191131506438186730397810806368517930378109939801855401198146020012241708454526590940672114387451626365242287279016559120153407419391811060195436769803350806644845587338005187189045113857)

In [6]:
def crypt_rsa(msg, public_key):
    n, e = public_key
    num_chunks = 1
    
    while True:
        M_chunks = break_msg(msg, num_chunks)
        for chunk in M_chunks:
            if toint(chunk) >= n:
                num_chunks += 1
                break
        else:
            return map(lambda m: pow(toint(m), e, n), M_chunks)  # return a map object which needs to be iterated to get all values. This saves memory.
        
encrypted_msg = crypt_rsa(msg, public)

In [7]:
for chunk in encrypted_msg:
    print(chunk)

34838993476127272364835782202842636025594803334530969370981019061591190351968216710544942983374820778436023611592069327989900736436669234559594636200925105866472312350487848263862314718077316782087595561403154902407566783259733796029033910216172780724922906067841837763975529097182397727302561235642214085369


In [8]:
def decrypt_rsa(chunk, private_key):
    p, q, d = private_key
    n = p * q
    decrypted = pow(chunk, d, n)
    return tostr(decrypted)

In [10]:
decrypt_rsa(chunk, private)

'I am legend is such a great movie starring Will Smith. I love watching him.'


<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercice 11.6: <span style="color:red">!!!</span> Casser le chiffrement RSA

Voici une clef publique

In [55]:
def quadratic_residue(n, k):
    """ List all quadratic residues of n that are less than k """
    qr = []
    primes = primes_lte(k)
    for prime in primes:
        for i in range(1, prime):
            if pow(i, 2, prime) == n%prime:
                qr.append(prime)
                break
    return qr

quadratic_residue(87463, 23)

[2, 3, 13, 17, 19]

In [15]:
public = (73722206893746878039310298412768333517547506486427363913406174815240823284857,
          33921003048397584579835360477549223828723590186917811609938274008840181968499)
n, e = public

et un message chiffré avec cette clef publique:

In [35]:
[22973877239788873882837788687834740958145091979501565167824992597825600406974,
 48503379361942356829127901273483580639426474600801539865525830360302350602689,
 2224798454942012298637628855810886175704245737423608868190613249161861526055,
 4500720701216145302036625979462397411127541711596515042635302843142748047486,
 35445000935671280429079877747553363121645781096430386417428307485228904386825,
 48627712259501563035806415688912560481020577805784144969245386699539463833735,
 71868389092768589092947834441169271512227882469129366706758279960751502739157,
 26019603019482382085505727901092092122241438959147654068117475447783602572984,
 65039729472521706954510624828984329309712256390395416184704490683377874857302,
 11805320141319662342135552217286927868466795280631816945032387836622798495117]

**a)**
Décrypter le message précédent !

<!-- --- begin hint in exercise --- -->

**Indication:**
Il est probablement nécessaire de choisir un algorithme adapté, par exemple basé sur des cribles quadratiques (QS, MPQS, SIQS).

<!-- --- end hint in exercise --- -->

<!-- --- end exercise --- -->